In [13]:
import utils as ut

import requests
import time
import random
import json
import pandas as pd
import pickle
import re
import folium
import matplotlib.pyplot as plt
import os
import time

from bs4 import BeautifulSoup
from urllib.request import urlopen
from fake_useragent import UserAgent 
from datetime import date
from geopy.geocoders import Nominatim
from selenium import webdriver

import warnings
warnings.filterwarnings('ignore')

In [14]:
df_sky = pd.read_excel('df_skyscanners_clean.xlsx')
#via spazi prima e dopo valore
df_sky['AIRLINE'] = df_sky['AIRLINE'].str.strip()
#tutto in minuscolo
df_sky['AIRLINE'] = df_sky['AIRLINE'].str.lower()
#togli punteggiatura
df_sky['AIRLINE'] = df_sky['AIRLINE'].str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True)
#normalizza valori togliendo accenti
df_sky['AIRLINE'] = df_sky['AIRLINE'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

print (type('AIRLINE'))
#df_sky.head()
#1387 righe

# Trova e mostra i duplicati
duplicati = df_sky[df_sky.duplicated()]
print("Duplicati trovati:", len(duplicati))
print(duplicati)

# Rimuovi i duplicati e mostra il DataFrame senza duplicati
df_no_dup_sky = df_sky.drop_duplicates()
print("\nDataFrame senza duplicati:")
#print(df_senza_duplicati)
df_no_dup_sky.head()

print("\nOra il DataFrame ha:", len(df_no_dup_sky), "righe - (in origine erano 1387)")

<class 'str'>
Duplicati trovati: 6
          AIRLINE
207       airlink
209      airnorth
485   eastafrican
1134   smartwings
1312      vivaair
1348    world2fly

DataFrame senza duplicati:

Ora il DataFrame ha: 1381 righe - (in origine erano 1387)


In [15]:
df_no_dup_sky.to_excel("sky.xlsx", index=False)

In [53]:
df_af = pd.read_excel('df_airfleets_clean.xlsx')

#duplico l'intero df
df_af['AIRLINE_copy'] = df_af['AIRLINE'].copy()
df_af['COUNTRY_copy'] = df_af['COUNTRY'].copy()
df_af['INFORMATION/N.AIRCRAFT_copy'] = df_af['INFORMATION/N.AIRCRAFT'].copy()

#via spazi
df_af['AIRLINE'] = df_af['AIRLINE'].str.strip()
#tutto in minuscolo
df_af['AIRLINE'] = df_af['AIRLINE'].str.lower()
#togli punteggiatura
df_af['AIRLINE'] = df_af['AIRLINE'].str.replace(r'[^\w\s]', '', regex=True).str.replace(r'\s+', '', regex=True)
#normalizza valori togliendo accenti
df_af['AIRLINE'] = df_af['AIRLINE'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

print (type('AIRLINE'))
#df_af.head()
#4031 righe

# Trova e mostra i duplicati
#duplicati = df_af[df_af.duplicated()]
duplicati = df_af[df_af.duplicated(subset=['AIRLINE', 'COUNTRY', 'INFORMATION/N.AIRCRAFT'])]
print("\nDuplicati trovati:", len(duplicati))
print(duplicati)

#Rimuovi i duplicati e mostra il DataFrame senza duplicati
df_no_dup_af = df_af.drop_duplicates()
print("\nDataFrame senza duplicati:")
#print(df_senza_duplicati)
df_no_dup_af.head()

print("\nOra il DataFrame ha:", len(df_no_dup_af), "righe - (in origine erano 4031)")

<class 'str'>

Duplicati trovati: 2
          AIRLINE  COUNTRY              INFORMATION/N.AIRCRAFT AIRLINE_copy  \
1823  germanwings  Germany  inactive (with supported aircraft)  Germanwings   
3472    sultanair   Turkey  inactive (with supported aircraft)    Sultanair   

     COUNTRY_copy         INFORMATION/N.AIRCRAFT_copy  
1823      Germany  inactive (with supported aircraft)  
3472       Turkey  inactive (with supported aircraft)  

DataFrame senza duplicati:

Ora il DataFrame ha: 4031 righe - (in origine erano 4031)


In [5]:
df_no_dup_af.to_excel("af.xlsx", index=False)

In [54]:
risultato = pd.merge(df_no_dup_af, df_no_dup_sky, on='AIRLINE', how='inner')

conteggio_compagnie = risultato2['AIRLINE'].value_counts()
compagnie_con_piu_di_1_occorrenza = conteggio_compagnie[conteggio_compagnie >= 1]

#Visualizza le compagnie aeree e il relativo conteggio
print(compagnie_con_piu_di_1_occorrenza)


AIRLINE
jettime            2
easternairlines    2
airitaly           2
germanwings        2
flyme              2
                  ..
cubana             1
cyprusairways      1
daalloairlines     1
dalianairlines     1
zambiaairways      1
Name: count, Length: 840, dtype: int64


In [62]:
#delete colonne originali
risultato.drop(columns=['AIRLINE', 'COUNTRY', 'INFORMATION/N.AIRCRAFT'], inplace=True)

#rename le colonne copiate eliminando "_copy"
risultato.rename(columns={'AIRLINE_copy': 'AIRLINE', 'COUNTRY_copy': 'COUNTRY', 'INFORMATION/N.AIRCRAFT_copy': 'INFORMATION/N.AIRCRAFT'}, inplace=True)

print(risultato)

              AIRLINE      COUNTRY              INFORMATION/N.AIRCRAFT
0     Aegean Airlines       Greece                                  59
1          Aer Lingus      Ireland                                  54
2      Aero Caribbean         Cuba                                   2
3    Aero Contractors      Nigeria                                   1
4           Aero Dili  Timor-Leste                                   1
..                ...          ...                                 ...
847     Yeti Airlines        Nepal                                   5
848   Zagros Airlines         Iran                                  12
849         Zagrosjet         Iraq  inactive (with supported aircraft)
850  Zambezi Airlines       Zambia  inactive (with supported aircraft)
851    Zambia Airways       Zambia                                   1

[852 rows x 3 columns]


In [61]:
risultato.to_excel("risultato.xlsx", index=False)

In [65]:
df_inactive = risultato[risultato['INFORMATION/N.AIRCRAFT'].str.contains('inactive')]

#Estrai 5 righe casuali da df_inactive
#random_state se specificato estrai gli stessi valori in run successivi
righe_casuali = df_inactive.sample(n=5, random_state=42)  

nuovo_df = pd.DataFrame(righe_casuali)

# Stampa il nuovo DataFrame
print(nuovo_df)

         AIRLINE          COUNTRY              INFORMATION/N.AIRCRAFT
522     Luke Air            Malta  inactive (with supported aircraft)
588   Open Skies   United Kingdom  inactive (with supported aircraft)
386  Germanwings          Germany  inactive (with supported aircraft)
690       Skyjet  Antigua Barbuda  inactive (with supported aircraft)
103    Air Sinai            Egypt  inactive (with supported aircraft)
